# Topic Modeling of Product Reviews on Amazon Scraped using Selenium in Python
> A tutorial demonstrating scraping product reviews from Amazon and extracting and analysing the topics from the text data.

- toc: true
- comments: true
- categories: [python, text analytics, web scraping, topic modeling, selenium, gensim, nlp]

## Introduction

The blog covers the step-by-step process to scrap product reviews from Amazon webpage and analysing main topics from the extracted data. We will scrap 1000 reviews from the Amazon for Apple iPhone 11 64GB. With this data, we will convert each review doc into bag-of words for applying the topic modeling algorithm. We will be using Latent Dirichlet Allocation (LDA) algorithm in this tutorial. The main python libraries used are:

- selenium: Selenium is a portable framework for testing web applications. We will be using this to interact with the browser and open URLs (https://pypi.org/project/selenium/)
- gensim: Gensim is an open-source library for unsupervised topic modeling and natural language processing, using modern statistical machine learning (https://pypi.org/project/gensim/)

## Web Scraping

Web scraping is a technique for extracting information from the internet automatically using a software that simulates human web surfing. Web scraping helps us extract large volumes of data about customers, products, people, stock markets, etc. It is usually difficult to get this kind of information on a large scale using traditional data collection methods. We can utilize the data collected from a website such as e-commerce portal, social media channels to understand customer behaviors and sentiments, buying patterns, and brand attribute associations which are critical insights for any business.

The first and foremost thing while scraping a website is to understand the structure of the website. We will be scraping the reviews for Apple iPhone 11 64GB on Amazon.in website. We will scrape 1000 reviews from different users across multiple pages. We will scrape user name, date of review and review and export it into a .csv file for any further analysis.

### Import Packages

- Instll selenium package (if not already worked with before) using command '!pip install selenium'
- Import webdriver from selenium in the notebook which we use to open an instance of Chrome browser
- The executable file for launching Chrome 'chromedriver.exe' should be in the same folder as the notebook

In [1]:
#Importing packages
from selenium import webdriver
import pandas as pd

### Script for Scraping

The below code opens the new chrome browser window and open our website with the url link provided. By the way, chrome knows that you are accessing it through an automated software!

In [2]:
driver = webdriver.Chrome('chromedriver.exe')
url = 'https://www.amazon.in/Apple-iPhone-11-64GB-White/product-reviews/B07XVMCLP7/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber1'
driver.get(url)

Woha! We just opened an url from python notebook.

![](img/chrome.png)

We will inspect 3 items (user id, date and comment) on our web page and understand how we can extract them.
- Xpath for User id: Inspecting the userid, we can see the highlighted text represents the XML code for user id.The XML path (XPath)for the userid is shown below:
    
    //*[@id="customer_review-RBOIMRTKIYBBR"]/div[1]/a/div[2]/span
    
![](img/chrome-2.png)
    
There is an interesting thing to note here that the XML path contains a review id, which uniquely denotes each review on the website. This will be very helpful as we try to recursively scrape multiple comments.

- Xpath for Date & review: Similarily, we will find the XPaths for date and review. 
- Selenium has a function called “find_elements_by_xpath”. We will pass our XPath into this function and get a selenium element. Once we have the element, we can extract the text inside our XPath using the ‘text’ function.
- We will recursively run the code for different review id and extract user id, date and review for each review id. Also, we will recursively go to next pages by simply changing the page numbers in the url to extract more comments until we get the desired number of comments.

In [4]:
driver = webdriver.Chrome('chromedriver.exe')

#Creating empty data frame to store user_id, dates and comments from ~5K users.
data = pd.DataFrame(columns = ['date','username','review'])

j = 1
while (j<=130):
    # Running while loop only till we get 1K reviews
    if (len(data)<1000):
        url = 'https://www.amazon.in/Apple-iPhone-11-64GB-White/product-reviews/B07XVMCLP7/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=' + str(j)
        driver.get(url)
        ids = driver.find_elements_by_xpath("//*[contains(@id,'customer_review-')]")
        review_ids = []
        for i in ids:
            review_ids.append(i.get_attribute('id'))

        for x in review_ids:
            #Extract dates from for each user on a page
            date_element = driver.find_elements_by_xpath('//*[@id="' + x +'"]/span')[0]
            date = date_element.text

            #Extract user ids from each user on a page
            username_element = driver.find_elements_by_xpath('//*[@id="' + x +'"]/div[1]/a/div[2]/span')[0]
            username = username_element.text

            #Extract Message for each user on a page
            review_element = driver.find_elements_by_xpath('//*[@id="' + x +'"]/div[4]')[0]
            review = review_element.text
            
           #Adding date, userid and comment for each user in a dataframe    
            data.loc[len(data)] = [date,username,review]
        j=j+1
    else:
        break

### Data Cleaning

- We perform few data cleaning operations such as replacing line breaks with a space and copy the data into .csv file which can be used for further analysis.

In [5]:
import copy
data = copy.deepcopy(data)

def remove_space(s):
    return s.replace("\n"," ")

data['review'] = data['review'].apply(remove_space)
data.to_csv('amazon_reviews.csv', header=True, sep=',')

In [6]:
data = pd.read_csv('amazon_reviews.csv',index_col=[0])
data

,date,username,review
0,Reviewed in India on 20 October 2019,Suman Biswas,May be my first negative review about the prod...
1,Reviewed in India on 17 September 2019,Kaushik Bajaj,It's very expensive but the quality you get is...
2,Reviewed in India on 29 September 2019,Sunny Kumar,The iPhone design is good and the camera quali...
3,Reviewed in India on 30 September 2019,shanu Kumar,Awesome Phone. Nice upgrade from iPhone 6s to ...
4,Reviewed in India on 14 October 2019,Amazon Customer,My Phone is Producing Too Much Heat Even Didn’...
...,...,...,...
995,Reviewed in India on 4 March 2020,Md.Imran,Rt
996,Reviewed in India on 1 March 2020,Amazon Customer,❤️
997,Reviewed in India on 9 March 2020,Chirag Patel,Ok
998,Reviewed in India on 11 March 2020,chintu,Excellent


- Since the goal of further analysis is to perform topic modeling, we will solely focus on the review text, and drop other metadata columns i.e. date and user name.

In [7]:
# Remove the columns
data = data.drop(columns=['date', 'username'], axis=1)
# Print out the data
data

,review
0,May be my first negative review about the prod...
1,It's very expensive but the quality you get is...
2,The iPhone design is good and the camera quali...
3,Awesome Phone. Nice upgrade from iPhone 6s to ...
4,My Phone is Producing Too Much Heat Even Didn’...
...,...
995,Rt
996,❤️
997,Ok
998,Excellent


## Topic Modeling using LDA

Topic modeling is a type of statistical modeling for discovering the abstract “topics” that occur in a collection of documents. Latent Dirichlet Allocation (LDA) is an example of topic model and is used to classify text in a document to a particular topic. LDA is a generative probabilistic model that assumes each topic is a mixture over an underlying set of words, and each document is a mixture of over a set of topic probabilities.

Illustration of LDA input/output workflow (Credit: http://chdoig.github.io/pytexas2015-topic-modeling/#/3/4)

![](img/lda-1.png)

### Data Pre-processing

We will preprocess the review data using gensim library. Few of the actions performed by preprocess_string as follows:
- Tokenization: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
- All stopwords are removed.
- Words are lemmatized: words in third person are changed to first person and verbs in past and future tenses are changed into present. 
- Words are stemmed: words are reduced to their root form.

Please see below the output after pre-processing one of the reviews.

In [10]:
import gensim
from gensim.parsing.preprocessing import preprocess_string

# print unprocessed text
print(data.review[1])

# print processed text
print(preprocess_string(data.review[1]))

It's very expensive but the quality you get is osum
['expens', 'qualiti', 'osum']


In [11]:
processed_data = data['review'].map(preprocess_string)
processed_data

0      [neg, review, product, amazon, elat, receiv, i...
1                                [expens, qualiti, osum]
2      [iphon, design, good, camera, qualiti, awesom,...
3      [awesom, phone, nice, upgrad, iphon, iphon, lo...
4      [phone, produc, heat, didn’t, sim, half, hour,...
                             ...                        
995                                                   []
996                                                   []
997                                                   []
998                                              [excel]
999                                              [excel]
Name: review, Length: 1000, dtype: object

### Preparign Document-Term-Matrix

Gensim requires that tokens be converted to a dictionary. In this instance a dictionary is a mapping between words and their integer IDs. We then create a  Document-Term-Matrix where we use Bag-of-Words approach returning the vector of word and its frequency (number of occurences in the document) for each document.

In [12]:
# Importing Gensim
import gensim
from gensim import corpora

# Creating the term dictionary of our list of documents (corpus), where every unique term is assigned an index. 
dictionary = corpora.Dictionary(processed_data)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in processed_data]

### Running LDA Model

We will now run the LDA Model. The number of topics you give is largely a guess/arbitrary. The model assumes the document contains that many topics. However, finding the number of topics explaining the data is a optimisation problem and can be found by 'Coherence Model'.

Here, we have used number of topics = 3

In [14]:
#RUN THE MODEL
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
TOPIC_CNT= 3
ldamodel = Lda(doc_term_matrix, num_topics=TOPIC_CNT, id2word = dictionary, passes=50)

We can then see the weights of top 20 words in each topic, which can help us to explain the topic.

In [16]:
#Results
topics= ldamodel.print_topics(num_topics=TOPIC_CNT, num_words=20)
topics

[(0,
  '0.050*"phone" + 0.031*"iphon" + 0.019*"best" + 0.015*"amazon" + 0.015*"nice" + 0.014*"great" + 0.013*"camera" + 0.012*"time" + 0.012*"appl" + 0.010*"charger" + 0.009*"product" + 0.008*"bui" + 0.008*"charg" + 0.007*"qualiti" + 0.007*"us" + 0.006*"good" + 0.006*"deliveri" + 0.006*"perfect" + 0.006*"look" + 0.006*"batteri"'),
 (1,
  '0.032*"phone" + 0.026*"awesom" + 0.024*"iphon" + 0.015*"love" + 0.011*"great" + 0.010*"io" + 0.010*"dai" + 0.010*"camera" + 0.009*"android" + 0.008*"displai" + 0.008*"it’" + 0.008*"batteri" + 0.007*"screen" + 0.007*"appl" + 0.007*"review" + 0.006*"pro" + 0.006*"best" + 0.006*"experi" + 0.006*"feel" + 0.006*"word"'),
 (2,
  '0.066*"good" + 0.041*"batteri" + 0.036*"camera" + 0.035*"phone" + 0.032*"product" + 0.025*"life" + 0.023*"qualiti" + 0.023*"appl" + 0.018*"iphon" + 0.018*"best" + 0.014*"awesom" + 0.014*"bui" + 0.014*"amaz" + 0.013*"monei" + 0.011*"excel" + 0.011*"valu" + 0.011*"perform" + 0.010*"mobil" + 0.009*"great" + 0.009*"worth"')]

### Extracting Topics

We can identify the follow topics emerging out of reviews of Amazon iPhone 11 64GB:

- Topic #1: There seems to discussion of heat/ charging issue with the product.
- Topic #2: The discussion on iPhone's features such as camera, display, battery.
- Topic #3: iPhone being value for money and discussion on Amazon delivery service.

In [17]:
word_dict = {};
for i in range(TOPIC_CNT):
    words = ldamodel.show_topic(i, topn = 20)
    word_dict['Topic #' + '{:2d}'.format(i+1)] = [i[0] for i in words]
pd.DataFrame(word_dict)

,Topic # 1,Topic # 2,Topic # 3
0,phone,phone,good
1,iphon,awesom,batteri
2,best,iphon,camera
3,amazon,love,phone
4,nice,great,product
5,great,io,life
6,camera,dai,qualiti
7,time,camera,appl
8,appl,android,iphon
9,charger,displai,best


The below code provide the % of topic a document is about. This helps to find the dominant topic in each review. 

In [18]:
doc_to_topic = []
for i in range(len(doc_term_matrix)):
    top_topics = ldamodel.get_document_topics(doc_term_matrix[i], minimum_probability=0.0)
    topic_vec = [top_topics[j][1] for j in range(TOPIC_CNT)]
    doc_to_topic.append(topic_vec)

In [19]:
#Dataframe of topic
document_topics = pd.DataFrame(doc_to_topic)
document_topics = document_topics.rename(columns=lambda x: x + 1)
document_topics.columns = document_topics.columns.astype(str)
document_topics = document_topics.rename(columns=lambda x: 'Topic #' + x)

In [20]:
#Dataframe of review and topics
data_new = pd.concat([data,document_topics],axis=1,join='inner')
data_new.head()

,review,Topic #1,Topic #2,Topic #3
0,May be my first negative review about the prod...,0.990397,0.004789,0.004815
1,It's very expensive but the quality you get is...,0.084920,0.089119,0.825960
2,The iPhone design is good and the camera quali...,0.989838,0.004876,0.005286
3,Awesome Phone. Nice upgrade from iPhone 6s to ...,0.455081,0.503810,0.041108
4,My Phone is Producing Too Much Heat Even Didn’...,0.978437,0.010337,0.011225


## Endnotes

I hope this blog helps in understanding how powerful Topic Modeling is in understanding unstructured textual data.  Feel free to play around with the code by opening in Colab or cloning the repo in github.

If you have any comments or suggestions please comment below or reach out to me at - [Twitter](https://twitter.com/rahulsingla0959) or [LinkedIn](https://www.linkedin.com/in/rahul-singla1/)